# Process monitoring

> Process monitoring module.

In [ ]:
#| default_exp monitors.process

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import psutil

In [ ]:
#| export
def get_process_info(top_n:int=5):
    """Get top processes by CPU and memory usage."""
    processes = []

    # Get all processes with their info
    for proc in psutil.process_iter(['pid', 'name', 'cpu_percent', 'memory_percent', 'memory_info', 'username', 'status']):
        try:
            pinfo = proc.info
            # Skip kernel threads and processes with 0% CPU and memory
            if pinfo['cpu_percent'] is not None and (pinfo['cpu_percent'] > 0 or pinfo['memory_percent'] > 0):
                processes.append({
                    'pid': pinfo['pid'],
                    'name': pinfo['name'],
                    'cpu_percent': pinfo['cpu_percent'] or 0,
                    'memory_percent': pinfo['memory_percent'] or 0,
                    'memory_mb': pinfo['memory_info'].rss / (1024 * 1024) if pinfo['memory_info'] else 0,
                    'username': pinfo['username'] if pinfo['username'] else 'N/A',
                    'status': pinfo['status'] if pinfo['status'] else 'unknown'
                })
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            continue

    # Sort by CPU usage and get top N
    top_cpu = sorted(processes, key=lambda x: x['cpu_percent'], reverse=True)[:top_n]

    # Sort by memory usage and get top N
    top_memory = sorted(processes, key=lambda x: x['memory_percent'], reverse=True)[:top_n]

    # Get total process count
    total_processes = len(processes)

    # Get process status counts
    status_counts = {}
    for proc in processes:
        status = proc['status']
        status_counts[status] = status_counts.get(status, 0) + 1

    return {
        'top_cpu': top_cpu,
        'top_memory': top_memory,
        'total': total_processes,
        'status_counts': status_counts
    }

In [ ]:
get_process_info()

{'top_cpu': [{'pid': 1,
   'name': 'systemd',
   'cpu_percent': 0,
   'memory_percent': 0.014269958552766296,
   'memory_mb': 13.42578125,
   'username': 'root',
   'status': 'sleeping'},
  {'pid': 623,
   'name': 'systemd-journald',
   'cpu_percent': 0,
   'memory_percent': 0.017566539027277915,
   'memory_mb': 16.52734375,
   'username': 'root',
   'status': 'sleeping'},
  {'pid': 717,
   'name': 'systemd-udevd',
   'cpu_percent': 0,
   'memory_percent': 0.007506571156066762,
   'memory_mb': 7.0625,
   'username': 'root',
   'status': 'sleeping'},
  {'pid': 1218,
   'name': 'systemd-oomd',
   'cpu_percent': 0,
   'memory_percent': 0.006784146719586885,
   'memory_mb': 6.3828125,
   'username': 'systemd-oom',
   'status': 'sleeping'},
  {'pid': 1270,
   'name': 'systemd-resolved',
   'cpu_percent': 0,
   'memory_percent': 0.011853573368678432,
   'memory_mb': 11.15234375,
   'username': 'systemd-resolve',
   'status': 'sleeping'}],
 'top_memory': [{'pid': 10233,
   'name': 'firefox',


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()